In [ ]:
import optparse
from pathlib import Path
import sys
# sys.path.append(str(Path(__file__).resolve().parent))
# # -- end fix path --
from functools import lru_cache
from multiprocessing import Pool, Lock
from string import punctuation
# import multiprocessing
# import Levenshtein
import numpy as np
# import spacy
import nltk
# import shutil
# import time
# import pickle
# import hashlib

In [13]:
import os
import shutil
from math import ceil

def move_subfolders(source, target, ratios):
    # Ensure target subdirectories exist
    for subdir in ['train', 'test', 'valid']:
        os.makedirs(os.path.join(target, subdir), exist_ok=True)

    # Get list of subdirectories in the source
    subdirs = [d for d in os.listdir(source) if os.path.isdir(os.path.join(source, d))]

    # Calculate the number of directories for each category
    total_dirs = len(subdirs)
    train_size = ceil(total_dirs * ratios['train'])
    test_size = ceil(total_dirs * ratios['test'])
    valid_size = total_dirs - train_size - test_size

    # Optionally randomize the directory list
    import random
    random.shuffle(subdirs)

    # Split directories
    train_dirs = subdirs[:train_size]
    test_dirs = subdirs[train_size:train_size + test_size]
    valid_dirs = subdirs[train_size + test_size:]

    # Function to move subdirectories
    def move_folder_contents(subdirs, split):
        for subdir in subdirs:
            source_dir = os.path.join(source, subdir)
            target_dir = os.path.join(target, split, subdir)
            os.makedirs(target_dir, exist_ok=True)
            for file in os.listdir(source_dir):
                src_file = os.path.join(source_dir, file)
                if 'Hindi' not in file and os.path.isfile(src_file):
                    shutil.copy(src_file, os.path.join(target_dir, file))
            # Remove the original directory if it's now empty (optional)
            # if not os.listdir(source_dir):
            #     os.rmdir(source_dir)

    # Move subdirectory contents to the respective splits
    move_folder_contents(train_dirs, 'train')
    move_folder_contents(test_dirs, 'test')
    move_folder_contents(valid_dirs, 'valid')

# Define source and target paths
source_path = '/Users/adhvik/Downloads/Adhvik/8th Semester/Natural Language Processing /Project/MILDSum_Simplified_Summaries_Splits/MILDSum_Dataset_Splits/MILDSum_Split_1'
target_path = '/Users/adhvik/Downloads/Adhvik/8th Semester/Natural Language Processing /Project/MILDSum_Simplified_Summaries_Splits/MILDSum_Dataset_Splits/processed_split1'

# Define split ratios
ratios = {'train': 0.6, 'test': 0.2, 'valid': 0.2}

# Run the function to move subfolders
move_subfolders(source_path, target_path, ratios)

In [14]:
# remove empty lines at the end of the files in taget folder and sub folders. print the folder name if there are empty lines between the text.

import os
import re

def remove_empty_lines(file_path):
    with open(file_path, 'rb') as file:
        lines = file.readlines()
    with open(file_path, 'wb') as file:
        # write in a single line
        for line in lines:
            if line.strip() != '':
                file.write(line)


def remove_empty_lines_in_folder(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            remove_empty_lines(file_path)
            with open(file_path, 'rb') as file:
                lines = file.readlines()
                for line in lines:
                    if line.strip() == '':
                        print(file_path)
                        break

# Define the target path
target_path = '/Users/adhvik/Downloads/Adhvik/8th Semester/Natural Language Processing /Project/MILDSum_Simplified_Summaries_Splits/MILDSum_Dataset_Splits/processed_split1'

# Run the function to remove empty lines
remove_empty_lines_in_folder(target_path)

In [16]:
import os

def concatenate_texts(base_dir, folder_name, file_names):
    for file_name in file_names:
        # Prepare the path to write the concatenated results
        output_file_path = os.path.join(base_dir, folder_name, f"combined_{file_name}")
        with open(output_file_path, 'w', encoding='utf-8') as outfile:
            # Walk through each subdirectory within the folder
            for subdir, dirs, files in os.walk(os.path.join(base_dir, folder_name)):
                file_to_concat = os.path.join(subdir, file_name)
                # Check if the file exists and is not the output file
                
                if os.path.isfile(file_to_concat):
                    print(f"Processing {file_to_concat}")
                    with open(file_to_concat, 'r', encoding='utf-8') as infile:
                        contents = infile.read()
                        # remove \n from the content
                        contents = contents.replace('\n', ' ')
                        # Write to the combined file with a newline after each content
                        outfile.write(contents + '\n')

def main():
    base_dir = '/Users/adhvik/Downloads/Adhvik/8th Semester/Natural Language Processing /Project/MILDSum_Simplified_Summaries_Splits/MILDSum_Dataset_Splits/processed_split1'
    sub_folders = ['test', 'train', 'valid']
    file_names = ['English_Summary.txt', 'Judgment_Text.txt']

    for folder in sub_folders:
        concatenate_texts(base_dir, folder, file_names)

if __name__ == "__main__":
    main()

Processing /Users/adhvik/Downloads/Adhvik/8th Semester/Natural Language Processing /Project/MILDSum_Simplified_Summaries_Splits/MILDSum_Dataset_Splits/processed_split1/test/1050/English_Summary.txt
Processing /Users/adhvik/Downloads/Adhvik/8th Semester/Natural Language Processing /Project/MILDSum_Simplified_Summaries_Splits/MILDSum_Dataset_Splits/processed_split1/test/2827/English_Summary.txt
Processing /Users/adhvik/Downloads/Adhvik/8th Semester/Natural Language Processing /Project/MILDSum_Simplified_Summaries_Splits/MILDSum_Dataset_Splits/processed_split1/test/364/English_Summary.txt
Processing /Users/adhvik/Downloads/Adhvik/8th Semester/Natural Language Processing /Project/MILDSum_Simplified_Summaries_Splits/MILDSum_Dataset_Splits/processed_split1/test/889/English_Summary.txt
Processing /Users/adhvik/Downloads/Adhvik/8th Semester/Natural Language Processing /Project/MILDSum_Simplified_Summaries_Splits/MILDSum_Dataset_Splits/processed_split1/test/1578/English_Summary.txt
Processing /